<a href="https://colab.research.google.com/github/neel26desai/cmpe297_timegpt_tabula_gnn/blob/main/DatasetCreationTabula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zhao-zilong/Tabula.git

Cloning into 'Tabula'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 61 (delta 30), reused 40 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (61/61), 54.05 KiB | 13.51 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import sys
import os

# Replace 'path/to/your/cloned/repo' with the actual path to the repository
repo_path = '/content/Tabula'
sys.path.append(os.path.abspath(repo_path))

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Specify the model name
model_name = "distilgpt2"

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Save the model locally
model.save_pretrained("./distilgpt2_model")
tokenizer.save_pretrained("./distilgpt2_model")

print("Model and tokenizer saved successfully.")


In [4]:
from tabula import Tabula
import pandas as pd

In [21]:
data = pd.read_csv("/content/Tabula/Real_Datasets/Insurance_compressed.csv")

In [22]:
data.head()

,age,sex,bi,children,sm,region,charges
0,18,0,53.13,0,0,0,1163.4627
1,22,0,52.58,1,1,0,44501.3982
2,23,0,50.38,1,0,0,2438.0552
3,58,0,49.06,0,0,0,11381.3254
4,46,1,48.07,2,0,3,9432.9253


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   int64  
 2   bi        1338 non-null   float64
 3   children  1338 non-null   int64  
 4   sm        1338 non-null   int64  
 5   region    1338 non-null   int64  
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(5)
memory usage: 73.3 KB


In [24]:
categorical_columns = ["sex", "children", "sm", "region"]
model = Tabula(llm='distilgpt2', experiment_dir = "insurance_training", batch_size=32, epochs=400, categorical_columns = categorical_columns)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model and tokenizer saved successfully.


In [25]:
import torch
import os


In [10]:
os.makedirs('./pretrained',exist_ok=True)

ensure you download the model from https://drive.google.com/file/d/1_YxelekxY5MXhgn93MYgsZEEfBYAy7h6/view and put it here

In [26]:
# model.model.load_state_dict(torch.load("/content/distilgpt2_model/model.safetensors"), strict=False)
model.model.load_state_dict(torch.load("/content/pretrained/adult_king_insurance_intrusion_covtype_pretrained.pt"), strict=False)

<ipython-input-26-27a0d48d6aa9>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load("/content/pretrained/adult_king_insurance_intrusion_c

<All keys matched successfully>

In [29]:
model.fit(data, conditional_col = data.columns[0])

/content/Tabula/tabula/tabula.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  tabula_trainer = TabulaTrainer(self.model, training_args, train_dataset=tabula_ds, tokenizer=self.tokenizer,
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.366400
1000,0.906200
1500,0.764100
2000,0.706600
2500,0.677200
3000,0.660300
3500,0.647600
4000,0.636200
4500,0.628200
5000,0.621000


In [31]:
os.makedirs('./adult_training',exist_ok=True)

In [32]:
torch.save(model.model.state_dict(), "./adult_training/model_400epoch.pt")

In [33]:
synthetic_data = model.sample(n_samples=1338)
synthetic_data.to_csv("insurance_400epoch.csv", index=False)

  0%|          | 0/1338 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
1398it [00:06, 226.08it/s]
/content/Tabula/tabula/tabula.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')
